# Spatial Regression

Follow along and extended from http://darribas.org/gds_scipy16/ipynb_md/08_spatial_regression.html

Having used spatial data to map San Francisco transport lines, compare air and walking distance, and create a network graph representation of San Francisco Muni stops, I wanted to return to a more quantitative way of analysis. I began to look into [spatial statistics](https://support.esri.com/en/other-resources/gis-dictionary/term/97a2c5ae-d8cb-476f-b72c-fda1314c27f4) as a means of bringing more science than art into how I analyse spatial data.

Following the notebook by Dani Arribas-Bel and Sergio Rey for the most part, I port the code to reflect the pysal 2.0.0 update, use San Francisco instead of Austin data, pick a different exogenous feature, and perform some extensions where I use multiple spatial weights and compare the spatial statistical method with a baseline regression with lat and lon features.

## Required Libraries

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pysal as ps
import geopandas as gpd

%matplotlib inline

/Users/JiaHer/anaconda3/lib/python3.6/site-packages/pysal/model/spvcm/abstracts.py:10: UserWarning: The `dill` module is required to use the sqlite backend fully.
  from .sqlite import head_to_sql, start_sql


## Data
Data source: http://insideairbnb.com/get-the-data.html

In [2]:
data_path = 'sf_listings.csv.gz'
sf_listings = pd.read_csv(data_path)
sf_listings.head(1)

,id,listing_url,scrape_id,last_scraped,name,summary,space,description,experiences_offered,neighborhood_overview,...,instant_bookable,is_business_travel_ready,cancellation_policy,require_guest_profile_picture,require_guest_phone_verification,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
0,958,https://www.airbnb.com/rooms/958,20190201155620,2019-02-01,"Bright, Modern Garden Unit - 1BR/1B",Our bright garden unit overlooks a grassy back...,"Newly remodeled, modern, and bright garden uni...",Our bright garden unit overlooks a grassy back...,none,*Quiet cul de sac in friendly neighborhood *St...,...,t,f,moderate,f,f,1,1,0,0,1.54


In [3]:
print(sf_listings.columns.values.tolist())

['id', 'listing_url', 'scrape_id', 'last_scraped', 'name', 'summary', 'space', 'description', 'experiences_offered', 'neighborhood_overview', 'notes', 'transit', 'access', 'interaction', 'house_rules', 'thumbnail_url', 'medium_url', 'picture_url', 'xl_picture_url', 'host_id', 'host_url', 'host_name', 'host_since', 'host_location', 'host_about', 'host_response_time', 'host_response_rate', 'host_acceptance_rate', 'host_is_superhost', 'host_thumbnail_url', 'host_picture_url', 'host_neighbourhood', 'host_listings_count', 'host_total_listings_count', 'host_verifications', 'host_has_profile_pic', 'host_identity_verified', 'street', 'neighbourhood', 'neighbourhood_cleansed', 'neighbourhood_group_cleansed', 'city', 'state', 'zipcode', 'market', 'smart_location', 'country_code', 'country', 'latitude', 'longitude', 'is_location_exact', 'property_type', 'room_type', 'accommodates', 'bathrooms', 'bedrooms', 'beds', 'bed_type', 'amenities', 'square_feet', 'price', 'weekly_price', 'monthly_price', '

## Data Preprocessing
There are a few things being done here:
1. Identify the data features we would like to run a regression on.
2. Identify an exogenous feature and encode it as a binary integer value.
3. Slice the dataset we want - with the data features from step 1 - from the original data file.
4. Change the dependent variable from a string to a non-null float.
5. Create a spatial weight matrix based on coordinate values.

In [4]:
# Here is a list of features I would like to retain
retain = ['bathrooms', 'bedrooms', 'beds', 'guests_included']

In [5]:
# used to identify whether the listing has a patio or balcony
# by checking for the phrase Patio or balcony in the amenities column
def has_patio(a):
    if 'Patio or balcony' in a:
        return 1
    else:
        return 0
    
sf_listings['patio_balcony'] = sf_listings['amenities'].apply(has_patio)

In [6]:
# take a slice of the dataframe (of features we want to retain) and drop any empty values
dataset = sf_listings.loc[:, retain + ['patio_balcony', 'price']].dropna()
dataset.head()

,bathrooms,bedrooms,beds,guests_included,patio_balcony,price
0,1.0,1.0,2.0,2,0,$170.00
1,1.0,2.0,3.0,2,0,$235.00
2,4.0,1.0,1.0,1,0,$65.00
3,4.0,1.0,1.0,1,0,$65.00
4,1.5,2.0,2.0,2,0,$785.00


In [7]:
# make the price a float and push it onto a log scale
price = np.log(dataset['price'].apply(lambda x: float(x.strip('$').replace(',', '')))+ 0.000001)
price.head()

0    5.135798
1    5.459586
2    4.174387
3    4.174387
4    6.665684
Name: price, dtype: float64

In [8]:
# spatial weights matrix that connects every observation to its 8 nearest neighbors.
# This will allow us to get extra diagnostics from the baseline model.
w = ps.lib.weights.KNN(sf_listings.loc[dataset.index, ['longitude', 'latitude']].values, k=8)

# row standerdised = A weights matrix is row-standardized when the values of each of its rows sum to one.
w.transform = 'R'
w

/Users/JiaHer/anaconda3/lib/python3.6/site-packages/pysal/lib/weights/weights.py:170: UserWarning: The weights matrix is not fully connected. There are 2 components
  warnings.warn("The weights matrix is not fully connected. There are %d components" % self.n_components)


## Baseline Regression
This is a basic Ordinary Least Squares Regression. The weight matrix is included here to include spatial dependency tests in the output.

In [9]:
# https://pysal.readthedocs.io/en/v1.12.0/library/spreg/ols.html
baseline_regression = ps.model.spreg.OLS(y = price.values[:, None],
                                         x = dataset.drop('price', axis=1).values,
                                         w = w,
                                         spat_diag = True,
                                         name_x = dataset.drop('price', axis=1).columns.tolist(),
                                         name_y = 'ln(price)') 

In [10]:
print(baseline_regression.summary)

REGRESSION
----------
SUMMARY OF OUTPUT: ORDINARY LEAST SQUARES
-----------------------------------------
Data set            :     unknown
Weights matrix      :     unknown
Dependent Variable  :   ln(price)                Number of Observations:        7173
Mean dependent var  :      5.0780                Number of Variables   :           6
S.D. dependent var  :      0.7106                Degrees of Freedom    :        7167
R-squared           :      0.3093
Adjusted R-squared  :      0.3088
Sum squared residual:    2501.573                F-statistic           :    641.7902
Sigma-square        :       0.349                Prob(F-statistic)     :           0
S.E. of regression  :       0.591                Log likelihood        :   -6400.012
Sigma-square ML     :       0.349                Akaike info criterion :   12812.024
S.E of regression ML:      0.5905                Schwarz criterion     :   12853.292

-----------------------------------------------------------------------------

## Spatially lagged exogenous regressors
The exogenous feature here is the presence of patios and balconies. The number of patios or balconies in the vicinity can be calculated using a non row standardised weight matrix.

In [11]:
w_patio = ps.lib.weights.KNN(sf_listings.loc[dataset.index,['longitude', 'latitude']].values, k=8)
dataset_w = dataset.assign(w_patio=ps.lib.weights.lag_spatial(w_patio, dataset['patio_balcony'].values))

/Users/JiaHer/anaconda3/lib/python3.6/site-packages/pysal/lib/weights/weights.py:170: UserWarning: The weights matrix is not fully connected. There are 2 components
  warnings.warn("The weights matrix is not fully connected. There are %d components" % self.n_components)


In [12]:
exogenous_regressors = ps.model.spreg.OLS(y = price.values[:, None],
                                          x = dataset_w.drop('price', axis=1).values,
                                          w=w,
                                          spat_diag=True,
                                          name_x=dataset_w.drop('price', axis=1).columns.tolist(),
                                          name_y='ln(price)')

In [13]:
print(exogenous_regressors.summary)

REGRESSION
----------
SUMMARY OF OUTPUT: ORDINARY LEAST SQUARES
-----------------------------------------
Data set            :     unknown
Weights matrix      :     unknown
Dependent Variable  :   ln(price)                Number of Observations:        7173
Mean dependent var  :      5.0780                Number of Variables   :           7
S.D. dependent var  :      0.7106                Degrees of Freedom    :        7166
R-squared           :      0.3093
Adjusted R-squared  :      0.3087
Sum squared residual:    2501.548                F-statistic           :    534.7682
Sigma-square        :       0.349                Prob(F-statistic)     :           0
S.E. of regression  :       0.591                Log likelihood        :   -6399.975
Sigma-square ML     :       0.349                Akaike info criterion :   12813.950
S.E of regression ML:      0.5905                Schwarz criterion     :   12862.097

-----------------------------------------------------------------------------

## Spatially lagged endogenous regressors
The endogenous data feature here is the price. The argument is that people tend to price their listings based in part on what the going rate is around them. Crucially however, the price is also the dependent variable. Running OLS on it would violate its assumptions. GM_Lag is a method that addresses this issue.

In [14]:
endogenous_regressors = ps.model.spreg.GM_Lag(y = price.values[:, None],
                                              x = dataset.drop('price', axis=1).values,
                                              w = w,
                                              spat_diag = True,
                                              name_x = dataset.drop('price', axis=1).columns.tolist(),
                                              name_y = 'ln(price)'
                                             ) 

In [15]:
print(endogenous_regressors.summary)

REGRESSION
----------
SUMMARY OF OUTPUT: SPATIAL TWO STAGE LEAST SQUARES
--------------------------------------------------
Data set            :     unknown
Weights matrix      :     unknown
Dependent Variable  :   ln(price)                Number of Observations:        7173
Mean dependent var  :      5.0780                Number of Variables   :           7
S.D. dependent var  :      0.7106                Degrees of Freedom    :        7166
Pseudo R-squared    :      0.2819
Spatial Pseudo R-squared:  0.3104

------------------------------------------------------------------------------------
            Variable     Coefficient       Std.Error     z-Statistic     Probability
------------------------------------------------------------------------------------
            CONSTANT       4.9187567       0.2327048      21.1373265       0.0000000
           bathrooms      -0.0515667       0.0101851      -5.0629730       0.0000004
            bedrooms       0.3145885       0.0123935      2

## Results

In [16]:
from sklearn.metrics import mean_squared_error as mse

mses = pd.Series({'OLS': mse(price, baseline_regression.predy.flatten()),
                  'OLS+Patio': mse(price, exogenous_regressors.predy.flatten()),
                  'Price': mse(price, endogenous_regressors.predy_e)
                    })
mses.sort_values()

Price        0.348159
OLS+Patio    0.348745
OLS          0.348749
dtype: float64

# Extensions

* What if the baseline regression was run with coordinates as a data feature?
* How about if we did a spatial regression with both patio and price?
* Is there a different exogenous feature that might be a larger factor than patio/balcony?

## Lat + Lon Regression

In [17]:
lat_lon_data = dataset
lat_lon_data['longitude'] = sf_listings['longitude']
lat_lon_data['latitude'] = sf_listings['latitude']

In [18]:
lat_lon_data.head()

,bathrooms,bedrooms,beds,guests_included,patio_balcony,price,longitude,latitude
0,1.0,1.0,2.0,2,0,$170.00,-122.433856,37.769310
1,1.0,2.0,3.0,2,0,$235.00,-122.421018,37.745112
2,4.0,1.0,1.0,1,0,$65.00,-122.452505,37.766690
3,4.0,1.0,1.0,1,0,$65.00,-122.451828,37.764872
4,1.5,2.0,2.0,2,0,$785.00,-122.436374,37.775249


In [19]:
lat_lon_regression = ps.model.spreg.OLS(y = price.values[:, None],
                                   x = lat_lon_data.drop('price', axis=1).values,
                                   w = w,
                                   spat_diag = True,
                                   name_x = lat_lon_data.drop('price', axis=1).columns.tolist(),
                                   name_y = 'ln(price)'
                                  ) 

In [20]:
print(lat_lon_regression.summary)

REGRESSION
----------
SUMMARY OF OUTPUT: ORDINARY LEAST SQUARES
-----------------------------------------
Data set            :     unknown
Weights matrix      :     unknown
Dependent Variable  :   ln(price)                Number of Observations:        7173
Mean dependent var  :      5.0780                Number of Variables   :           8
S.D. dependent var  :      0.7106                Degrees of Freedom    :        7165
R-squared           :      0.3771
Adjusted R-squared  :      0.3765
Sum squared residual:    2255.745                F-statistic           :    619.7853
Sigma-square        :       0.315                Prob(F-statistic)     :           0
S.E. of regression  :       0.561                Log likelihood        :   -6029.026
Sigma-square ML     :       0.314                Akaike info criterion :   12074.052
S.E of regression ML:      0.5608                Schwarz criterion     :   12129.077

-----------------------------------------------------------------------------

## Price + Patio

In [21]:
w = ps.lib.weights.KNN(sf_listings.loc[dataset.index, ['longitude', 'latitude']].values, k=8)
w

/Users/JiaHer/anaconda3/lib/python3.6/site-packages/pysal/lib/weights/weights.py:170: UserWarning: The weights matrix is not fully connected. There are 2 components
  warnings.warn("The weights matrix is not fully connected. There are %d components" % self.n_components)


In [22]:
price_patio_regressor = ps.model.spreg.GM_Lag(y = price.values[:, None],
                                              x = dataset_w.drop('price', axis=1).values,
                                              w = w,
                                              spat_diag = True,
                                              name_x = dataset_w.drop('price', axis=1).columns.tolist(),
                                              name_y = 'ln(price)'
                                             ) 

In [23]:
print(price_patio_regressor.summary)

REGRESSION
----------
SUMMARY OF OUTPUT: SPATIAL TWO STAGE LEAST SQUARES
--------------------------------------------------
Data set            :     unknown
Weights matrix      :     unknown
Dependent Variable  :   ln(price)                Number of Observations:        7173
Mean dependent var  :      5.0780                Number of Variables   :           8
S.D. dependent var  :      0.7106                Degrees of Freedom    :        7165
Pseudo R-squared    :      0.2790
Spatial Pseudo R-squared:  0.3106

------------------------------------------------------------------------------------
            Variable     Coefficient       Std.Error     z-Statistic     Probability
------------------------------------------------------------------------------------
            CONSTANT       4.9590909       0.2412081      20.5593835       0.0000000
           bathrooms      -0.0515494       0.0102068      -5.0505130       0.0000004
            bedrooms       0.3143237       0.0124205      2

## Transit

In [24]:
# Pulls out a number of phrases that would indicate the presence of a nearby transit stop
# Assigns it as a binary value for regression analysis

transit_types = ['Bus', 'bus', 'muni', 'Muni', 'BART', 'Bart', 'bart']

sf_listings['transit'] = sf_listings['transit'].fillna('N/A')

def has_transit(a):
    if any(transit_type in a for transit_type in transit_types):
        return 1
    else:
        return 0

sf_listings['transit'] = sf_listings['transit'].apply(has_transit)

In [25]:
dataset_w_transit = sf_listings.loc[:, retain + ['transit', 'price']].dropna()
dataset_w_transit.head()

,bathrooms,bedrooms,beds,guests_included,transit,price
0,1.0,1.0,2.0,2,1,$170.00
1,1.0,2.0,3.0,2,0,$235.00
2,4.0,1.0,1.0,1,1,$65.00
3,4.0,1.0,1.0,1,1,$65.00
4,1.5,2.0,2.0,2,0,$785.00


In [26]:
w_transit = ps.lib.weights.KNN(sf_listings.loc[dataset.index,['longitude', 'latitude']].values, k=8)
dataset_w_transit = dataset.assign(w_transit=ps.lib.weights.lag_spatial(w_transit, dataset_w_transit['transit'].values))

/Users/JiaHer/anaconda3/lib/python3.6/site-packages/pysal/lib/weights/weights.py:170: UserWarning: The weights matrix is not fully connected. There are 2 components
  warnings.warn("The weights matrix is not fully connected. There are %d components" % self.n_components)


In [27]:
transit_regressor = ps.model.spreg.OLS(y = price.values[:, None],
                               x = dataset_w_transit.drop('price', axis=1).values,
                               w=w,
                               spat_diag=True,
                               name_x=dataset_w_transit.drop('price', axis=1).columns.tolist(),
                               name_y='ln(price)'
                              )

In [28]:
print(transit_regressor.summary)

REGRESSION
----------
SUMMARY OF OUTPUT: ORDINARY LEAST SQUARES
-----------------------------------------
Data set            :     unknown
Weights matrix      :     unknown
Dependent Variable  :   ln(price)                Number of Observations:        7173
Mean dependent var  :      5.0780                Number of Variables   :           9
S.D. dependent var  :      0.7106                Degrees of Freedom    :        7164
R-squared           :      0.3780
Adjusted R-squared  :      0.3773
Sum squared residual:    2252.649                F-statistic           :    544.2122
Sigma-square        :       0.314                Prob(F-statistic)     :           0
S.E. of regression  :       0.561                Log likelihood        :   -6024.101
Sigma-square ML     :       0.314                Akaike info criterion :   12066.202
S.E of regression ML:      0.5604                Schwarz criterion     :   12128.105

-----------------------------------------------------------------------------

## Final Results

In [29]:
from sklearn.metrics import mean_squared_error as mse

mses = pd.Series({'OLS': mse(price, baseline_regression.predy.flatten()),
                  'OLS+Patio': mse(price, exogenous_regressors.predy.flatten()),
                  'Price': mse(price, endogenous_regressors.predy_e),
                  'Lat+Lon': mse(price, lat_lon_regression.predy.flatten()),
                  'Price+Patio': mse(price, price_patio_regressor.predy_e),
                  'Transit': mse(price, transit_regressor.predy.flatten())
                    })
mses.sort_values()

Transit        0.314046
Lat+Lon        0.314477
Price+Patio    0.348096
Price          0.348159
OLS+Patio      0.348745
OLS            0.348749
dtype: float64